# Data processing

This notebook can be used for processing various data formats found in this folder. 

Some of these datasets are already somewhat edited by hand before being edited here 
(e.g. reformatting to consistent columns). But as minimally as possible.

#1. EIA data
#2. CAT data portal data  
*  Limited data available (only 20 countries), worth using?

In [3]:
# import modules

# system
import re
import sys
import os

# data handling
import pandas as pd
import numpy as np

# doesn't work on windows? and has some weird dependencies that are difficult to install on all platforms. 
#from countrynames import to_code_3


# open climate data packages
from countrygroups import UNFCCC, EUROPEAN_UNION, ANNEX_ONE, NON_ANNEX_ONE
from shortcountrynames import to_name

# global stocktake tools
import gst_tools.gst_utils as utils



In [22]:
# 1 EIA Energy data

# !!! Currently retains all available countries because EIA data has country names and no ISO codes !!!

# some of the data from the EIA has been pre-processed to an easy to read .csv file. However, a 
# bit more processing is needed to generate separate .csv files for each variable. That is 
# performed by this section of the notebook. It does not need to be repeated but is retained
# here for documentation. 

raw_data_file = "EIA-International_data-energy-production-consumption-by-country.csv"

# first available year is 1980, but more data available later
start_year = 1990

# Based on countrygroups package, select the group of countries you would like to extract. 
# Note that the raw data may also include groups.
needed_countries = UNFCCC
new_source_name = 'EIA'

# get the data
fname = os.path.join('', 'input-data', raw_data_file)
print('reading ' + fname)
raw_data = pd.read_csv(fname)
new_data = raw_data.dropna()

# rename some columns
#new_data = raw_data.rename(columns={'countryISO': 'country'})

# reduce the countries or regions to only those desired
# and tell the user which ones are being removed

# rename countries to ISO-3 codes
#for country in new_data['country']: countrynames.to_code_3(country)
new_data['country'] = new_data['country'].apply(to_code_3)
all_countries = new_data['country'].unique()
removed_countries = list(set(all_countries) - set(needed_countries))
if removed_countries:
    print('Some countries being trimmed from dataset:')
    for country in removed_countries:
        if country:
            print('   ' + to_name(country))
    print('---------')
new_data = new_data.loc[new_data['country'].isin(needed_countries)]

# tell the user if any of the needed countries are missing and, if yes, which ones:
missing_countries = list(set(needed_countries) - set(new_data['country'].unique()))
if missing_countries:
    print('Not all countries requested were available in the raw data. You are missing the following:')
    for country in missing_countries:
        print('   ' + to_name(country))
    print('---------')
    
# Check for available variables and sectors
variables = new_data['variable'].unique()
fuels = new_data['fuel'].unique()
  
# make a new file with each one...

for var in variables:
    for fuel in fuels:
        
        print('getting data for ' + var + ' and ' + fuel)
        
        data_selected = new_data.loc[(new_data['variable'] == var) &
                                     (new_data['fuel'] == fuel)]
        
        # Check the data format
        if not utils.verify_data_format(data_selected):

            print('WARNING: The data is not correctly formatted! Please check your input data and processing!')

        else:

            # define the variable name
            new_variable_name = (var + '-' + fuel)
            new_variable_name = new_variable_name.replace(' ', '-').lower()
            data_selected['variable'] = new_variable_name
            
            # make nans were appropriate
            data_selected = data_selected.replace('(s)','nan')
            data_selected = data_selected.replace('--','nan')
            
            data_selected = utils.change_first_year(data_selected, start_year)
            
            # make column names strings
            data_selected.columns = data_selected.columns.astype(str)
            
            # define filename as composite of variable and source name
            fname_out = new_source_name + '_' + new_variable_name + '.csv' 
            fullfname_out = os.path.join('proc-data', fname_out)

            # check folder exists
            if not os.path.exists('proc-data'):
                os.makedirs('proc-data')

            # write to csv in proc data folder
            data_selected.to_csv(fullfname_out, index=False)

            # celebrate success 
            print('Processed data written to file! - ' + fullfname_out)


# reduce to only required years
#new_data = utils.change_first_year(new_data, start_year)

# make the columns strings
data_selected

reading input-data\EIA-International_data-energy-production-consumption-by-country.csv


NameError: name 'to_code_3' is not defined

In [17]:
# 2. CAT data

raw_data_file = "CAT_data_portal_20191011.csv"

new_source_name = 'CAT-decarb'

def convert_ISO2_to_ISO3(iso2code):

    """
    convert a country 2-letter ISO code to a 3-letter ISO code.
    """

    country_code_file = "gst_tools/country_codes.csv"
    country_codes = pd.read_csv(country_code_file)

    iso3code = country_codes.loc[country_codes['ISO2']==iso2code, 'ISO3']
    
    return iso3code
    
    
# get the data
fname = os.path.join('', 'input-data', raw_data_file)
print('reading ' + fname)
raw_data = pd.read_csv(fname)
new_data = raw_data.copy()
new_data = new_data.drop('label', axis=1)
new_data = new_data.rename({'country':'ISO2'}, axis=1)

# Get the ISO3 country codes!
country_code_file = "gst_tools/country_codes.csv"
country_codes = pd.read_csv(country_code_file)

combined_data = pd.merge(new_data, country_codes, how='inner', on='ISO2')
combined_data = combined_data.rename({'ISO3':'country'}, axis=1)


# other pre-processing..



# only select a few of the available indicators? 

# Check if all of the needed countries are missing and, if yes, which ones:
needed_countries = UNFCCC
missing_countries = list(set(needed_countries) - set(combined_data['country'].unique()))
if missing_countries:
    print('Not all countries requested were available in the raw CAT data. You are missing the following:')
    for country in missing_countries:
        print('   ' + to_name(country))
    print('---------')



reading input-data\CAT_data_portal_20191011.csv
Not all countries requested were available in the raw CAT data. You are missing the following:
   Tanzania
   Monaco
   Nauru
   St. Vincent & Grenadines
   Panama
   Slovenia
   Spain
   Bangladesh
   Niue
   Bahamas
   Greece
   Estonia
   Trinidad & Tobago
   Ireland
   Netherlands
   Timor-Leste
   Kuwait
   Kiribati
   Togo
   Rwanda
   Guinea
   Sri Lanka
   St. Kitts & Nevis
   Myanmar
   Vietnam
   Albania
   Mozambique
   Israel
   Central African Republic
   Liechtenstein
   Guinea-Bissau
   Nicaragua
   Croatia
   Comoros
   Pakistan
   Malaysia
   Tonga
   Jamaica
   Portugal
   Eritrea
   Iran
   Burkina Faso
   Antigua & Barbuda
   Bolivia
   Papua New Guinea
   Denmark
   Cyprus
   Turkmenistan
   Seychelles
   Bosnia & Herzegovina
   Serbia
   Gabon
   Guyana
   Laos
   Mongolia
   Tuvalu
   Eswatini
   Moldova
   Liberia
   Czechia
   Poland
   Qatar
   Uganda
   Lesotho
   Cambodia
   Tunisia
   Honduras
   Senegal
   Pa

In [20]:

new_data.indicator.unique()

array(['Oil and Gas activity: production',
       'Electricity activity (per capita)',
       'Electricity emissions intensity',
       'Share of renewable electricity generation',
       'Renewable energy generation (index)', 'EVs per capita',
       'Forest land area index',
       'Agriculture activity (meat): consumption',
       'Agriculture activity (meat): production',
       'Waste generation (per capita)', 'Emissions per capita',
       'GDP per capita', 'Emissions intensity of GDP',
       'Cement activity: production per capita',
       'Cement emissions intensity',
       'Steel Activity: production per capita',
       'Modal split: share of private passenger transport',
       'Road transport emissions intensity',
       'Road transport biofuel/electrification share',
       'Buildings activity (residential)',
       'Buildings activity (commercial)',
       'Buildings emissions intensity (per capita)',
       'Agriculture activity (total): consumption',
       'Primary en

In [21]:
# testing


